## 0: Save intermediate data

In [1]:
import os
import pandas as pd
import pyreadstat
from collections import defaultdict

In [2]:
# Read in metadata
field_selection_path = '../metadata/guatemala_variable_filtered_v4.csv'
field_df = pd.read_csv(field_selection_path)

# Include only selected variables
included_df = field_df[field_df['included'].str.lower() == 'yes']

# Create mapping diictionary between modules and associated variables
module_var_map = defaultdict(list)
for _, row in included_df.iterrows():
    module_name = row['module'].strip().upper()
    variable_name = row['variable_name'].strip()
    module_var_map[module_name].append(variable_name)

module_var_map.items()

dict_items([('HOGARES.DTA', ['region', 'area', 'hogar', 'thogar', 'factor']), ('AUTOIDH.DTA', ['hogar', 'idenho_1', 'getnicoh']), ('ECV02H01.DTA', ['hogar', 'item', 'p01b04a', 'p01b04b', 'p01b04c']), ('CONSUMO5.DTA', ['depto', 'mupio', 'sector', 'hogar', 'upm2', 'agreg3']), ('ECV18N15.DTA', ['hogar', 'p15b01', 'p15b02']), ('ECV21A16.DTA', ['hogar', 'tipo', 'tcuerda', 'p16a07a', 'p16a07b']), ('ECV31A16.DTA', ['hogar', 'tipo', 'item', 'p16j02', 'p16j03']), ('ECV17E14.DTA', ['hogar', 'tipo', 'item', 'p14a01', 'p14a02']), ('ECV09P05.DTA', ['hogar', 'caso', 'sexo', 'edad', 'p05a02', 'p05a03', 'p05b05', 'p07b01', 'p07b06b', 'p07b29']), ('ECV28A16.DTA', ['hogar', 'tipo', 'item', 'p16g01', 'p16g02']), ('ECV01H01.DTA', ['hogar', 'p01a01', 'p01a02', 'p01a03', 'p01a04', 'p01a05a', 'p01a05b', 'p01a05c', 'p01a05d', 'p01a05e', 'p01a05f', 'p01a06', 'p01a07', 'p01a08', 'p01a09', 'p01a10', 'p01a11', 'p01a12', 'p01a13', 'p01a14', 'p01a23', 'p01a25', 'p01a26', 'p01a34', 'p01a35', 'p01a39', 'p01a44', 'p01

In [3]:
# Read the corresponding fields of each module
dataframes = {} 
raw_data_path = '../raw_data_downloaded'

for module_file, var_list in module_var_map.items():

    module_path = os.path.join(raw_data_path, module_file)
    
    if not os.path.exists(module_path):
        print(f"Warning: {module_file} not found in raw_data/")
        continue
    
    # Read modules in old stata format using pd.read_stata
    if module_file in ['ECOM02.DTA','ECOM03.DTA']:
        df = pd.read_stata(module_path, columns=var_list, convert_categoricals=True)
    else: # Read modules in stata version 110 (not supported by read_stata, use pyreadstat)
        df, meta = pyreadstat.read_dta(module_path, usecols=var_list, apply_value_formats=True, encoding='iso-8859-1')

    # Rename hhid 
    if 'hogar' in df.columns:
        df['hogar'] = df['hogar'].astype(str)
        df.rename(columns={'hogar': 'hhid'}, inplace=True)

    """ 
    # Check type of 
    if 'caso' in df.columns:
        df['caso'] = df['caso'].astype(str)
    """
    
    dataframes[module_file] = df
    print(f"Loaded {module_file} with {len(var_list)} variables.")

output_path = '../intermediate_data'
os.makedirs(output_path, exist_ok=True)

for module_file, df in dataframes.items():
    csv_name = module_file.replace('.DTA', '.csv')
    df.to_csv(os.path.join(output_path, csv_name), index=False, encoding='utf-8-sig')

Loaded HOGARES.DTA with 5 variables.
Loaded AUTOIDH.DTA with 3 variables.
Loaded ECV02H01.DTA with 5 variables.
Loaded CONSUMO5.DTA with 6 variables.
Loaded ECV18N15.DTA with 3 variables.
Loaded ECV21A16.DTA with 5 variables.
Loaded ECV31A16.DTA with 5 variables.
Loaded ECV17E14.DTA with 5 variables.
Loaded ECV09P05.DTA with 10 variables.
Loaded ECV28A16.DTA with 5 variables.
Loaded ECV01H01.DTA with 29 variables.
Loaded ECV19N15.DTA with 2 variables.
Loaded ECOM03.DTA with 6 variables.
Loaded ECV11P10.DTA with 7 variables.
Loaded ECOM02.DTA with 5 variables.


## 1: Clean ```HOGARES``` - Main table

In [4]:
hogares = pd.read_csv('../intermediate_data/HOGARES.csv', encoding='utf-8-sig')

# Check for missing data
missing_report = hogares.isnull().sum()
print('Missing report: ')
print(missing_report)

# Rename columns 
hogares = hogares.rename(columns={'factor': 'hh_wgt', 'thogar': 'hh_size'})

# Convert data types
hogares['hhid'] = hogares['hhid'].astype('int64').astype('string')
hogares['region'] = hogares['region'].astype('category')
hogares['area'] = hogares['area'].astype('category')

# save csv
hogares.to_csv('../cleaned_data/HOGARES_cleaned.csv', index=False, encoding='utf-8-sig')

print('\nData Types: ')
print(hogares.dtypes)

hogares

Missing report: 
region    0
area      0
hhid      0
thogar    0
factor    0
dtype: int64

Data Types: 
region           category
area             category
hhid       string[python]
hh_size           float64
hh_wgt            float64
dtype: object


,region,area,hhid,hh_size,hh_wgt
0,metropolitana,urbana,1,4.0,541.0
1,metropolitana,urbana,2,3.0,541.0
2,metropolitana,urbana,3,6.0,541.0
3,metropolitana,urbana,4,1.0,541.0
4,metropolitana,urbana,5,3.0,541.0
...,...,...,...,...,...
7271,suroriente,rural,7272,1.0,219.0
7272,suroriente,rural,7273,5.0,219.0
7273,suroriente,rural,7274,7.0,219.0
7274,suroriente,rural,7275,6.0,219.0


## 2. Clean ```AUTOIDH``` - Identification

In [5]:
autoidh = pd.read_csv('../intermediate_data/AUTOIDH.csv', encoding='utf-8-sig')
missing_report = autoidh.isnull().sum()

print("Missing report: ")
print(missing_report)

autoidh['idenho_1'] = autoidh['idenho_1'].astype('category')
autoidh['getnicoh'] = autoidh['getnicoh'].astype('category')

# save csv
autoidh.to_csv('../cleaned_data/AUTOIDH_cleaned.csv', index=False, encoding='utf-8-sig')

print('\nData Types: ')
print(autoidh.dtypes)

autoidh

Missing report: 
hhid        0
idenho_1    0
getnicoh    0
dtype: int64

Data Types: 
hhid           int64
idenho_1    category
getnicoh    category
dtype: object


,hhid,idenho_1,getnicoh
0,1,Otro Maya,Indigenas
1,2,No indigena,No Indigenas
2,3,No indigena,No Indigenas
3,4,No indigena,No Indigenas
4,5,No indigena,No Indigenas
...,...,...,...
7271,7272,No indigena,No Indigenas
7272,7273,No indigena,No Indigenas
7273,7274,No indigena,No Indigenas
7274,7275,No indigena,No Indigenas


## 3. Clean ```CONSUMO5``` - Consumption

In [6]:
consumo5 = pd.read_csv('../intermediate_data/CONSUMO5.csv', encoding='utf-8-sig')

missing_report = consumo5.isnull().sum()
print("Missing report: ")
print(missing_report)

# make agreg3 to be per capita per day in 2017 USD PPP
conversion_factor = 0.6878405536 
consumo5['consumption_per_capita_per_day'] = (consumo5['agreg3'] / 365) * conversion_factor

consumo5['hhid'] = consumo5['hhid'].astype('Int64').astype('string')

for col in ['mupio', 'sector', 'depto']:
    consumo5[col] = consumo5[col].astype('category')

consumo5.drop(columns = ['agreg3'], inplace = True)

consumo5.to_csv('../cleaned_data/CONSUMO5_cleaned.csv', index=False, encoding='utf-8-sig')

print('\nData Types: ')
print(consumo5.dtypes)

consumo5

Missing report: 
depto     0
mupio     0
sector    0
hhid      0
upm2      0
agreg3    0
dtype: int64

Data Types: 
depto                                   category
mupio                                   category
sector                                  category
hhid                              string[python]
upm2                                     float64
consumption_per_capita_per_day           float64
dtype: object


,depto,mupio,sector,hhid,upm2,consumption_per_capita_per_day
0,guatemala,3.0,3.0,1,1031003.0,22.024104
1,guatemala,3.0,3.0,2,1031003.0,20.392536
2,guatemala,3.0,3.0,3,1031003.0,21.341869
3,guatemala,3.0,3.0,4,1031003.0,112.619080
4,guatemala,3.0,3.0,5,1031003.0,61.555193
...,...,...,...,...,...,...
7271,jutiapa,13.0,7.0,7272,22132008.0,13.389319
7272,jutiapa,13.0,7.0,7273,22132008.0,4.574897
7273,jutiapa,13.0,7.0,7274,22132008.0,3.830335
7274,jutiapa,13.0,7.0,7275,22132008.0,4.126309


## 4. Clean ```ECV01H01``` - Housing characteristics

In [7]:
ecv01h01 = pd.read_csv('../intermediate_data/ECV01H01.csv', encoding='utf-8-sig')

# Check for columns with missing info
missing_report = ecv01h01.isnull().sum()
missing_col = missing_report[missing_report > 0].index.tolist()
print("Initial missing report:")
print(missing_report[missing_report > 0])

# Amend dtype hhid
ecv01h01['hhid'] = ecv01h01['hhid'].astype('Int64').astype('string')

# Since all columns with missing info are categorical,
# set as categorical dtype, add missing as category
for col in missing_col:
    ecv01h01[col] = ecv01h01[col].astype('category').cat.add_categories('missing').fillna('missing')

# Set type of numeric columns
int_cols = ['p01a06', 'p01a07', 'p01a08', 'p01a09']
for col in int_cols:
    ecv01h01[col] = ecv01h01[col].astype('Int64')

# Get all categorical columns and set as category dtype
cat_cols = list(set(ecv01h01.columns) - set(['hhid']) - set(missing_col) - set(int_cols))
for col in cat_cols:
    ecv01h01[col] = ecv01h01[col].astype('category')

ecv01h01.to_csv('../cleaned_data/ECV01H01_cleaned.csv', index=False, encoding='utf-8-sig')

print("\nFinal missing report:")
missing_report = ecv01h01.isnull().sum()
print(missing_report[missing_report > 0])

print('\nData Types: ')
print(ecv01h01.dtypes)

Initial missing report:
p01a11      47
p01a13    2185
p01a14    2185
p01a35    1975
p01a44    3019
p01a45    3019
dtype: int64

Final missing report:
Series([], dtype: int64)

Data Types: 
hhid       string[python]
p01a01           category
p01a02           category
p01a03           category
p01a04           category
p01a05a          category
p01a05b          category
p01a05c          category
p01a05d          category
p01a05e          category
p01a05f          category
p01a06              Int64
p01a07              Int64
p01a08              Int64
p01a09              Int64
p01a10           category
p01a11           category
p01a12           category
p01a13           category
p01a14           category
p01a23           category
p01a25           category
p01a26           category
p01a34           category
p01a35           category
p01a39           category
p01a44           category
p01a45           category
p01c01           category
dtype: object


## 5. Clean ```ECV02H01``` - Energy Source

In [8]:
ecv02h01 = pd.read_csv('../intermediate_data/ECV02H01.csv', encoding='utf-8-sig')

missing_report = ecv02h01.isnull().sum()
print("Initial missing report: ")
print(missing_report[missing_report > 0])

# Step 1: Translating labels
energy_translation = {
    "candelas y/o veladoras": "candles",
    "kerosene (gas corriente)": "kerosene",
    "gas propano": "propane_gas",
    "carbon": "charcoal",
    "baterias": "batteries",
    "electricidad": "electricity",
    "leña o palos": "firewood",
    "otra fuente de energia": "other_energy"
}

use_map = {
    'p01b04a': 'household',
    'p01b04b': 'cooking',
    'p01b04c': 'home_business',
    #'p01b04d': 'other'
}


# Step 2: Handle missing values
for col in use_map.keys():
    ecv02h01[col] = ecv02h01[col].fillna('missing')

print("\nFinal missing report:")
missing_report = ecv01h01.isnull().sum()
print(missing_report[missing_report > 0])

ecv02h01

Initial missing report: 
p01b04a    31593
p01b04b    31593
p01b04c    31593
dtype: int64

Final missing report:
Series([], dtype: int64)


,hhid,item,p01b04a,p01b04b,p01b04c
0,1.0,candelas y/o veladoras,missing,missing,missing
1,1.0,kerosene (gas corriente),missing,missing,missing
2,1.0,gas propano,si,no,no
3,1.0,carbon,missing,missing,missing
4,1.0,baterias,si,si,no
...,...,...,...,...,...
58203,7276.0,carbon,missing,missing,missing
58204,7276.0,baterias,si,si,no
58205,7276.0,electricidad,missing,missing,missing
58206,7276.0,leÑa o palos,si,no,no


In [9]:
# Step 3: wide to long
long_df = pd.melt(
    ecv02h01,
    id_vars=['hhid', 'item'],
    value_vars=list(use_map.keys()),
    var_name='use_code',
    value_name='value'
)

# Step 4: generate col names
long_df['energy'] = long_df['item'].map(energy_translation)
long_df['use'] = long_df['use_code'].map(use_map)
long_df['final_col'] = long_df['energy'] + '_' + long_df['use']

# Step 5: pivot to wide format
ecv02h01_cleaned = long_df.pivot_table(
    index='hhid',
    columns='final_col',
    values='value',
    aggfunc='first'
).reset_index()

# Step 6: hhid as string in case errors
ecv02h01_cleaned['hhid'] = ecv02h01_cleaned['hhid'].astype('Int64').astype('string')

ecv02h01_cleaned.to_csv('../cleaned_data/ECV02H01_cleaned.csv', index=False, encoding='utf-8-sig')

print('\nData Types: ')
print(ecv02h01_cleaned.dtypes)

ecv02h01_cleaned


Data Types: 
final_col
hhid                          string[python]
batteries_cooking                     object
batteries_home_business               object
batteries_household                   object
candles_cooking                       object
candles_home_business                 object
candles_household                     object
charcoal_cooking                      object
charcoal_home_business                object
charcoal_household                    object
electricity_cooking                   object
electricity_home_business             object
electricity_household                 object
kerosene_cooking                      object
kerosene_home_business                object
kerosene_household                    object
other_energy_cooking                  object
other_energy_home_business            object
other_energy_household                object
propane_gas_cooking                   object
propane_gas_home_business             object
propane_gas_household          

final_col,hhid,batteries_cooking,batteries_home_business,batteries_household,candles_cooking,candles_home_business,candles_household,charcoal_cooking,charcoal_home_business,charcoal_household,...,electricity_household,kerosene_cooking,kerosene_home_business,kerosene_household,other_energy_cooking,other_energy_home_business,other_energy_household,propane_gas_cooking,propane_gas_home_business,propane_gas_household
0,1,si,no,si,missing,missing,missing,missing,missing,missing,...,si,missing,missing,missing,missing,missing,missing,no,no,si
1,2,missing,missing,missing,missing,missing,missing,missing,missing,missing,...,si,missing,missing,missing,missing,missing,missing,no,no,si
2,3,no,no,si,missing,missing,missing,no,no,si,...,si,missing,missing,missing,missing,missing,missing,no,no,si
3,4,missing,missing,missing,missing,missing,missing,missing,missing,missing,...,si,missing,missing,missing,missing,missing,missing,no,no,si
4,5,no,no,si,no,si,no,no,no,si,...,si,missing,missing,missing,missing,missing,missing,no,no,si
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7271,7272,si,no,si,missing,missing,missing,missing,missing,missing,...,missing,no,no,si,missing,missing,missing,missing,missing,missing
7272,7273,si,no,si,missing,missing,missing,missing,missing,missing,...,missing,no,no,si,missing,missing,missing,missing,missing,missing
7273,7274,no,no,si,no,no,si,missing,missing,missing,...,missing,no,no,si,missing,missing,missing,missing,missing,missing
7274,7275,no,no,si,no,no,si,missing,missing,missing,...,missing,no,no,si,missing,missing,missing,missing,missing,missing


## 6. Clean ```ECV09P05``` - Demographic Characteristics

In [10]:
ecv09p05 = pd.read_csv('../intermediate_data/ECV09P05.csv', encoding='utf-8-sig')
missing_report = ecv09p05.isnull().sum()
print(missing_report[missing_report>0])

# formatting labels
ecv09p05['hhid'] = ecv09p05['hhid'].astype('int64').astype(str)
ecv09p05['caso'] = ecv09p05['caso'].astype('int64').astype(str)
ecv09p05.columns = ecv09p05.columns.str.lower()

ecv09p05

p05a03     11629
p05b05      8357
p07b01      8244
p07b06b    28687
p07b29     18000
dtype: int64


,hhid,caso,sexo,edad,p05a02,p05a03,p05b05,p07b01,p07b06b,p07b29
0,1,1,femenino,42.0,jefe (a),viudo (a),otro maya,lee y escribe,NaN,primaria
1,1,2,femenino,23.0,hijo (a),soltero (a),otro maya,lee y escribe,2.0,basicos
2,1,3,masculino,19.0,hijo (a),soltero (a),otro maya,lee y escribe,6.0,diversificado
3,1,4,femenino,0.0,nieto (a),NaN,NaN,NaN,NaN,NaN
4,2,1,masculino,60.0,jefe (a),unido (a),no indigena,lee y escribe,NaN,preparatoria
...,...,...,...,...,...,...,...,...,...,...
37766,7276,6,masculino,10.0,hijo (a),soltero (a),no indigena,no lee ni escribe,1.0,NaN
37767,7276,7,masculino,8.0,hijo (a),NaN,no indigena,no lee ni escribe,1.0,NaN
37768,7276,8,masculino,5.0,hijo (a),NaN,NaN,NaN,NaN,NaN
37769,7276,9,masculino,3.0,hijo (a),NaN,NaN,NaN,NaN,NaN


In [11]:
# Step 1: Count numbers of individuals in each household
ecv09p05['sexo'] = ecv09p05['sexo'].str.lower()

# Create boolean columns for each aggregation
ecv09p05['num_male'] = ecv09p05['sexo'] == 'masculino'
ecv09p05['num_female'] = ecv09p05['sexo'] == 'femenino'
ecv09p05['num_children'] = ecv09p05['edad'] <= 17
ecv09p05['num_adults'] = ecv09p05['edad'] >= 18
ecv09p05['num_elderly'] = ecv09p05['edad'] >= 60

# Aggregations
agg_dict = {
    'num_male': 'sum',
    'num_female': 'sum',
    'num_children': 'sum',
    'num_adults': 'sum',
    'num_elderly': 'sum',
}

hh_counts = ecv09p05.groupby('hhid').agg(agg_dict)

In [12]:
#Step 3: Extract the relevant information of the household head (based on p05a02 = 'jefe (a)')
ecv09p05['p05a02'] = ecv09p05['p05a02'].str.lower()
ecv09p05['p05a03'] = ecv09p05['p05a03'].str.lower()
ecv09p05['p05b05'] = ecv09p05['p05b05'].str.lower()

# Take out the row of the household head
hh_head_rows = ecv09p05[ecv09p05['p05a02'] == 'jefe (a)'].copy()

# Household head information
hh_head_info = hh_head_rows[['hhid', 'sexo', 'edad', 'p05a03', 'p05b05', 'p07b01', 'p07b29']].copy()
hh_head_info = hh_head_info.rename(columns={
    'sexo': 'hh_head_gender',
    'edad': 'hh_head_age',
    'p05a03': 'hh_head_marital_status',
    'p05b05': 'hh_head_ethnicity',
    'p07b01': 'hh_head_literacy',
    'p07b29': 'hh_head_education_level'
})

In [13]:
# Step 4: The person with the highest educational attainment in each household (educational level)
# Educational hierarchy ranking mapping
edu_rank = {
    'preparatoria': 1,
    'primaria': 2,
    'basicos': 3,
    'diversificado': 4,
    'universitario': 5,
    'post-grado': 6
}

ecv09p05['p07b29'] = ecv09p05['p07b29'].str.lower()
ecv09p05['edu_score'] = ecv09p05['p07b29'].map(edu_rank)

max_edu = ecv09p05.groupby('hhid').apply(
    lambda x: x.loc[x['edu_score'].idxmax(), 'p07b29'] if x['edu_score'].notna().any() else 'missing'
).reset_index(name='max_education_level')


C:\Users\PB\AppData\Local\Temp\ipykernel_7772\2649093771.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  max_edu = ecv09p05.groupby('hhid').apply(


In [14]:
# Step 5: The adult female with the highest educational attainment
adult_female = ecv09p05[(ecv09p05['sexo'] == 'femenino') & (ecv09p05['edad'] >= 18)].copy()
adult_female['edu_score'] = adult_female['p07b29'].map(edu_rank)

female_max_edu = adult_female.groupby('hhid').apply(
    lambda x: x.loc[x['edu_score'].idxmax(), 'p07b29'] if x['edu_score'].notna().any() else 'missing'
).reset_index(name='female_max_education_level')

C:\Users\PB\AppData\Local\Temp\ipykernel_7772\3415680403.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  female_max_edu = adult_female.groupby('hhid').apply(


In [15]:
# Step 6: The number of children currently enrolled (edad <= 17 and p07b06b not empty)
school_age_children = ecv09p05[(ecv09p05['edad'] <= 17) & (ecv09p05['p07b06b'].notna())]
num_enrolled_children = school_age_children.groupby('hhid').size().reset_index(name='num_children_enrolled')

In [16]:
# init household dataframe
from functools import reduce

dfs = [
    hh_counts,
    hh_head_info,
    max_edu,
    female_max_edu,
    num_enrolled_children
]

# Use reduce to merge all Dataframes (left join)
from functools import reduce
household_summary = reduce(lambda left, right: pd.merge(left, right, on='hhid', how='left'), dfs)

# Completion of missing values
household_summary['hh_head_education_level'] = household_summary['hh_head_education_level'].fillna('missing')
household_summary['female_max_education_level'] = household_summary['female_max_education_level'].fillna('missing')

count_vars = [
    'num_male', 'num_female', 'num_children', 'num_adults',
    'num_elderly', 'num_children_enrolled'
]

for col in count_vars:
    if col in household_summary.columns:
        household_summary[col] = household_summary[col].fillna(0).astype(int)

# Sort hhid by integer
household_summary['hhid_sort'] = household_summary['hhid'].astype(int)
household_summary = household_summary.sort_values('hhid_sort').drop(columns='hhid_sort')

household_summary['hhid'] = household_summary['hhid'].astype(str)
household_summary['hh_head_age'] = household_summary['hh_head_age'].astype(int).astype(str)

household_summary.to_csv('../cleaned_data/ECV09P05_cleaned.csv', index=False, encoding='utf-8-sig')


In [17]:
ecv09p05_cleaned = pd.read_csv('../cleaned_data/ECV09P05_cleaned.csv', encoding='utf-8-sig')
missing_report = ecv09p05_cleaned.isnull().sum()
print(missing_report[missing_report>0])
print(ecv09p05_cleaned.head())

Series([], dtype: int64)
   hhid  num_male  num_female  num_children  num_adults  num_elderly  \
0     1         1           3             1           3            0   
1     2         2           1             0           3            2   
2     3         5           1             2           4            0   
3     4         0           1             0           1            1   
4     5         2           1             1           2            0   

  hh_head_gender  hh_head_age hh_head_marital_status hh_head_ethnicity  \
0       femenino           42              viudo (a)         otro maya   
1      masculino           60              unido (a)       no indigena   
2      masculino           51             casado (a)       no indigena   
3       femenino           89              viudo (a)       no indigena   
4      masculino           49             casado (a)       no indigena   

  hh_head_literacy hh_head_education_level max_education_level  \
0    lee y escribe             

## 7. Clean ```ECV11P10``` - Household's living activities

In [18]:
ecv11p10 = pd.read_csv('../intermediate_data/ECV11P10.csv', encoding='utf-8-sig')
missing_report = ecv11p10.isnull().sum()
print(missing_report[missing_report>0])
print(ecv11p10.head())

p10d01     15024
p10d03     27023
p10d04     27020
p10d06     27023
p10e05a    31482
dtype: int64
   hhid  caso p10d01 p10d03 p10d04 p10d06 p10e05a
0   1.0   1.0    NaN    NaN    NaN    NaN     NaN
1   1.0   2.0     no    NaN    NaN    NaN     NaN
2   1.0   3.0     no    NaN    NaN    NaN     NaN
3   2.0   1.0     no    NaN    NaN    NaN     NaN
4   2.0   2.0    NaN    NaN    NaN    NaN     NaN


In [19]:
# data type
ecv11p10.columns = ecv11p10.columns.str.lower()
ecv11p10['hhid'] = ecv11p10['hhid'].astype('int64').astype(str)
ecv11p10['caso'] = ecv11p10['caso'].astype('int64').astype(str)
# Only retain the household head information (caso == '1')
hh_head_livelihood = ecv11p10[ecv11p10['caso'] == '1'].copy()


In [20]:
# Retain the specified field
selected_cols = ['hhid', 'p10d01', 'p10d03', 'p10d04', 'p10d06', 'p10e05a']
hh_head_livelihood = hh_head_livelihood[selected_cols]

# Handle missing values (fill 'missing' in all columns except hhid)
for col in hh_head_livelihood.columns:
    if col != 'hhid':
        hh_head_livelihood[col] = hh_head_livelihood[col].fillna('missing')

hh_head_livelihood.to_csv('../cleaned_data/ECV11P10_cleaned.csv', index=False, encoding='utf-8-sig')

In [21]:
ecv11p10_clean = pd.read_csv('../cleaned_data/ECV11P10_cleaned.csv', encoding='utf-8-sig')
missing_report = ecv11p10_clean.isnull().sum()
print(missing_report[missing_report>0])
print(ecv11p10_clean.head())

Series([], dtype: int64)
   hhid   p10d01   p10d03   p10d04   p10d06  p10e05a
0     1  missing  missing  missing  missing  missing
1     2       no  missing  missing  missing  missing
2     3       no  missing  missing  missing  missing
3     4  missing  missing  missing  missing  missing
4     5       no  missing  missing  missing  missing


In [22]:
full_hhids = pd.DataFrame({'hhid': [i for i in range(1, 7277)]})

df_full = full_hhids.merge(ecv11p10_clean, on='hhid', how='left')

# except for hhid，all missing value = 'missing'
for col in df_full.columns:
    if col != 'hhid':
        df_full[col] = df_full[col].fillna('missing')

df_full.to_csv('../cleaned_data/ECV11P10_cleaned.csv', index=False, encoding='utf-8-sig')

## 8. Clean ```ECV17E14``` - Home Equipment

In [23]:
ecv17e14 = pd.read_csv('../intermediate_data//ECV17E14.csv', encoding='utf-8-sig')
missing_report = ecv11p10.isnull().sum()
print(missing_report[missing_report>0])
print(ecv17e14.head())

p10d01     15024
p10d03     27023
p10d04     27020
p10d06     27023
p10e05a    31482
dtype: int64
   hhid                  tipo                       item p14a01  p14a02
0   1.0   articulos de cocina  estufa de gas / electrica     si     2.0
1   1.0   articulos de cocina        horno de microondas     no     NaN
2   1.0   articulos de cocina         horno convencional     no     NaN
3   1.0   articulos de cocina              refrigeradora     si     1.0
4   1.0   articulos de cocina         cafetera electrica     no     NaN


In [24]:
# Check the number of non-repetitive values in the "item" column
num_unique, unique_values = ecv17e14["item"].nunique(), ecv17e14["item"].unique()
print(num_unique, unique_values)

38 ['estufa de gas / electrica' 'horno de microondas' 'horno convencional'
 'refrigeradora' 'cafetera electrica' 'licuadora' 'exprimidor de jugos'
 'molino de nixtamal y otros' 'computadora personal' 'impresora'
 'camara fotografica' 'radio transistor' 'componente con cd'
 'grabadora / radiograbadora' 'maquina de escribir' 'televisor'
 'camara de video' 'video - camara / cassetera' 'otro de esparcimiento'
 'beeper' 'telefono' 'plancha electrica' 'plancha de brazas' 'lavadora'
 'secadora' 'ventilador' 'aspiradora' 'maquina de coser'
 'otro articulo del hogar' 'automovil' 'pick up' 'camionetilla'
 'moto o motoneta' 'bicicleta' 'carreta de bueyes' 'camion'
 'bote o lancha' 'otro vehiculo']


In [25]:
from collections import OrderedDict

# standardized column names are in lowercase
ecv17e14.columns = ecv17e14.columns.str.lower()
ecv17e14['hhid'] = ecv17e14['hhid'].astype('Int64')

# Handle the outlier p14a01 and generate the "Quantity of Items" column
ecv17e14['p14a01'] = ecv17e14['p14a01'].astype(str).str.lower()

def clean_quantity(row):
    if row['p14a01'] == 'no':
        return 0
    elif row['p14a01'] == 'si':
        return row['p14a02']
    else:
        return 0 

ecv17e14['p14a02'] = ecv17e14.apply(clean_quantity, axis=1)
ecv17e14['p14a02'] = ecv17e14['p14a02'].fillna(0).astype(int)

# Use OrderedDict to retain the translation order of items
item_translation = OrderedDict([
    ('estufa de gas / electrica', 'gas_stove'),
    ('horno de microondas', 'microwave'),
    ('horno convencional', 'oven'),
    ('refrigeradora', 'refrigerator'),
    ('cafetera electrica', 'coffee_maker'),
    ('licuadora', 'blender'),
    ('exprimidor de jugos', 'juicer'),
    ('molino de nixtamal y otros', 'nixtamal_mill'),
    ('computadora personal', 'computer'),
    ('impresora', 'printer'),
    ('camara fotografica', 'camera'),
    ('radio transistor', 'radio'),
    ('componente con cd', 'cd_player'),
    ('grabadora / radiograbadora', 'tape_recorder'),
    ('maquina de escribir', 'typewriter'),
    ('televisor', 'tv'),
    ('camara de video', 'video_camera'),
    ('video - camara / cassetera', 'cassette_camera'),
    ('otro de esparcimiento', 'other_entertainment'),
    ('beeper', 'beeper'),
    ('telefono', 'phone'),
    ('plancha electrica', 'electric_iron'),
    ('plancha de brazas', 'charcoal_iron'),
    ('lavadora', 'washer'),
    ('secadora', 'dryer'),
    ('ventilador', 'fan'),
    ('aspiradora', 'vacuum'),
    ('maquina de coser', 'sewing_machine'),
    ('otro articulo del hogar', 'other_household_item'),
    ('automovil', 'car'),
    ('pick up', 'pickup'),
    ('camionetilla', 'small_truck'),
    ('moto o motoneta', 'motorcycle'),
    ('bicicleta', 'bicycle'),
    ('carreta de bueyes', 'ox_cart'),
    ('camion', 'truck'),
    ('bote o lancha', 'boat'),
    ('otro vehiculo', 'other_vehicle')
])

# Add the translation column and the variable list
ecv17e14['item_en'] = ecv17e14['item'].map(item_translation)
ecv17e14['var_name'] = 'num_' + ecv17e14['item_en']

# pivot to wide format
ecv17e14_cleaned = ecv17e14.pivot_table(
    index='hhid',
    columns='var_name',
    values='p14a02',
    aggfunc='first'
).reset_index()


ecv17e14_cleaned['hhid_sort'] = ecv17e14_cleaned['hhid'].astype(int)
ecv17e14_cleaned = ecv17e14_cleaned.sort_values('hhid_sort').drop(columns='hhid_sort')

# Arrange the variable columns in the original order
ordered_cols = ['hhid'] + ['num_' + item_translation[item] for item in item_translation]
ecv17e14_cleaned = ecv17e14_cleaned[ordered_cols]

ecv17e14_cleaned.to_csv('../cleaned_data/ECV17E14_cleaned.csv', index=False, encoding='utf-8-sig')


## 9. Clean ```ECV18N15``` - Enterprise Control

In [26]:
ecv18n15 = pd.read_csv('../intermediate_data//ECV18N15.csv', encoding='utf-8-sig')
missing_report = ecv18n15.isnull().sum()
print(missing_report[missing_report>0])
print(ecv18n15.head())

p15b02    4624
dtype: int64
   hhid p15b01  p15b02
0   1.0     no     NaN
1   2.0     si     1.0
2   3.0     no     NaN
3   4.0     no     NaN
4   5.0     si     1.0


In [27]:
ecv18n15["p15b01"] = ecv18n15["p15b01"].str.lower()
inconsistent = ecv18n15[(ecv18n15["p15b01"] == "no") & (ecv18n15["p15b02"] > 0)]
print(inconsistent)

        hhid p15b01  p15b02
1446  1447.0     no     1.0
2239  2240.0     no     1.0


In [28]:
ecv18n15['hhid'] = ecv18n15['hhid'].astype('Int64').astype(str)
ecv18n15['p15b02'] = ecv18n15['p15b02'].fillna(0).astype(int).astype(str)
ecv18n15.to_csv('../cleaned_data/ECV18N15_cleaned.csv', index=False, encoding='utf-8-sig')


In [29]:
ecv18n15_clean = pd.read_csv('../cleaned_data/ECV18N15_cleaned.csv', encoding='utf-8-sig')
missing_report = ecv18n15_clean.isnull().sum()
print(missing_report[missing_report>0])
print(ecv18n15_clean.head())

Series([], dtype: int64)
   hhid p15b01  p15b02
0     1     no       0
1     2     si       1
2     3     no       0
3     4     no       0
4     5     si       1


## 10. Check ```ECV19N15``` - Business details - whether remain?

In [30]:
ecv19n15 = pd.read_csv('../intermediate_data//ECV19N15.csv', encoding='utf-8-sig')
missing_report = ecv19n15.isnull().sum()
print(missing_report[missing_report>0])
print(ecv19n15.head())

Series([], dtype: int64)
   hhid                    p15b04
0   2.0   industria manufacturera
1   5.0                  comercio
2   8.0                  comercio
3   9.0   industria manufacturera
4   9.0   industria manufacturera


In [31]:
# Check the number of non-repetitive values in the "item" column
num_unique, unique_values = ecv19n15["p15b04"].nunique(), ecv19n15["p15b04"].unique()
print(num_unique, unique_values)

12 [' industria manufacturera' ' comercio' ' servicios financieros'
 ' servicios de salud, sociales y personales' ' enseÑanza' ' construccion'
 ' agricultura, ganaderia, caza y pesca'
 ' transporte, almacenamiento y comunicaciones'
 ' administracion publica y defensa' ' actividad no especificada'
 ' electricidad, gas y agua' ' explotacion de minas y canteras']


Veriry first: For each hhid, is the p15b02 value (number of enterprises) in ecv18n15_cleaned equal to the number of rows (corresponding number of enterprise industries) of the hhid in ecv19n15?

In [32]:
import pandas as pd

ecv18n15 = pd.read_csv('../cleaned_data/ECV18N15_cleaned.csv', encoding='utf-8-sig')
ecv19n15 = pd.read_csv('../intermediate_data/ECV19N15.csv', encoding='utf-8-sig')

ecv18n15['hhid'] = ecv18n15['hhid'].astype(int)
ecv18n15['p15b02'] = ecv18n15['p15b02'].astype(int)
ecv19n15.columns = ecv19n15.columns.str.lower()
ecv19n15['hhid'] = ecv19n15['hhid'].astype(int)

# Only the families with business in ecv18n15 (p15b02 > 0) will be retained.
business_hh = ecv18n15[ecv18n15['p15b02'] > 0].copy()

In [33]:
# Count the number of industry items in ecv19n15
industry_counts = ecv19n15.groupby('hhid').size().reset_index(name='num_industries_reported')
# merge
validation_df = business_hh.merge(industry_counts, on='hhid', how='left')
validation_df['num_industries_reported'] = validation_df['num_industries_reported'].fillna(0).astype(int)

# Verify consistency
validation_df['match'] = validation_df['p15b02'] == validation_df['num_industries_reported']
mismatched = validation_df[validation_df['match'] == False]

if mismatched.empty:
    print("done")
else:
    print(f"❗ There are  {len(mismatched)} inconsistencies among families") 
    print(mismatched[['hhid', 'p15b02', 'num_industries_reported']])


❗ There are  10 inconsistencies among families
      hhid  p15b02  num_industries_reported
134    336       2                        1
140    346       2                        1
141    347       2                        1
549   1306       1                        0
566   1349       2                        1
1085  2763       2                        0
1086  2764       2                        0
1244  3120       3                        2
1726  4599       2                        1
2121  5641       1                        0


## 11. Clean ```ECV21A16``` - Amount of land owned

In [34]:
import pandas as pd

ecv21a16 = pd.read_csv('../intermediate_data/ECV21A16.csv', encoding='utf-8-sig')
missing_report = ecv21a16.isnull().sum()
print(missing_report[missing_report>0])
print(ecv21a16.head())


Series([], dtype: int64)
    hhid                         tipo                            tcuerda  \
0   28.0                     agricola  cuerda/tarea de 25 varas por lado   
1   31.0                     agricola  cuerda/tarea de 25 varas por lado   
2  297.0                     agricola  cuerda/tarea de 25 varas por lado   
3  363.0  mixta (agricola y pecuaria)  cuerda/tarea de 25 varas por lado   
4  378.0                     agricola  cuerda/tarea de 25 varas por lado   

   p16a07a  p16a07b  
0      1.0  manzana  
1     10.0   cuerda  
2     10.0  manzana  
3      2.0   cuerda  
4      5.0  manzana  


In [35]:
for col in ecv21a16.columns:
    unique_vals = ecv21a16[col].dropna().unique()
    print(f"Column: {col}")
    print(f"Unique values ({len(unique_vals)}): {unique_vals[:10]}")
    print()

Column: hhid
Unique values (2523): [ 28.  31. 297. 363. 378. 471. 472. 477. 478. 508.]

Column: tipo
Unique values (2): ['agricola' 'mixta (agricola y pecuaria)']

Column: tcuerda
Unique values (8): ['cuerda/tarea de 25 varas por lado' 'cuerda/tarea de 20 varas por lado'
 'cuerda/tarea de 30 varas por lado' 'cuerda/tarea de 40 varas por lado'
 'cuerda/tarea de 32 varas por lado' 'cuerda/tarea de 24 varas por lado'
 'cuerda/tarea de 28 varas por lado' 'cuerda/tarea de 50 varas por lado']

Column: p16a07a
Unique values (109): [ 1.  10.   2.   5.   3.  12.   1.5  7.   4.5  4. ]

Column: p16a07b
Unique values (5): ['manzana' 'cuerda' 'hectarea' 'caballeria' 'tarea']



In [36]:
import re
ecv21a16.columns = ecv21a16.columns.str.lower()
ecv21a16['hhid'] = ecv21a16['hhid'].astype(int)

# Extract the vara value ("xx varas" in tcuerda)
def extract_vara(val):
    match = re.search(r'(\d+)', val)
    if match:
        return int(match.group(1))
    return None

ecv21a16['vara_length'] = ecv21a16['tcuerda'].apply(extract_vara)

# Define the unit conversion function (return square meters)
def convert_to_sqm(row):
    value = row['p16a07a']
    unit = str(row['p16a07b']).strip().lower()
    vara = row['vara_length']
    
    if pd.isna(value) or value == '':
        return 0
    
    try:
        value = float(value)
    except:
        return 0

    if unit == 'manzana':
        return value * 6987
    elif unit == 'hectarea':
        return value * 10000
    elif unit == 'caballeria':
        return value * 78.58 * 10000
    elif unit in ['cuerda', 'tarea']:
        if pd.isna(vara):
            return 0
        else:
            vara_length_m = vara * 0.8421
            return value * (vara_length_m ** 2)
    else:
        return 0

# Apply conversion functions
ecv21a16['land_area_sqm'] = ecv21a16.apply(convert_to_sqm, axis=1)

# Apply conversion functions
land_area_by_hh = ecv21a16.groupby('hhid')['land_area_sqm'].sum().reset_index()

full_hhids = pd.DataFrame({'hhid': [i for i in range(1, 7277)]})

# Summarize the existing area according to hhid
land_area_by_hh = ecv21a16.groupby('hhid')['land_area_sqm'].sum().reset_index()
land_area_by_hh = full_hhids.merge(land_area_by_hh, on='hhid', how='left')
land_area_by_hh['land_area_sqm'] = land_area_by_hh['land_area_sqm'].fillna(0)

land_area_by_hh.to_csv('../cleaned_data/ECV21A16_cleaned.csv', index=False, encoding='utf-8-sig')


## 12. Clean ```ECV28A16``` - Agricultural equipment and facilities

In [74]:
ecv28a16 = pd.read_csv('../intermediate_data/ECV28A16.csv', encoding='utf-8-sig')
missing_report = ecv28a16.isnull().sum()
print(missing_report[missing_report>0])
print(ecv28a16.head())

p16g02    92052
dtype: int64
   hhid      tipo                        item p16g01  p16g02
0  28.0  agricola  implementos de tiro animal     no     NaN
1  28.0  agricola                     tractor     no     NaN
2  28.0  agricola    implementos para tractor     no     NaN
3  28.0  agricola                 cosechadora     no     NaN
4  28.0  agricola    sembradora o cultivadora     no     NaN


In [75]:
print(ecv28a16['item'].nunique(), ecv28a16['item'].unique())
print(ecv28a16['p16g01'].unique())
print(ecv28a16['p16g02'].unique())

26 ['implementos de tiro animal' 'tractor' 'implementos para tractor'
 'cosechadora' 'sembradora o cultivadora' 'bomba de agua'
 'camion, camioneta, jeep' 'animales de trabajo' 'bomba fumigadora'
 'planta electrica' 'equipo de riego' 'pequeÑas herramientas' 'ordeÑadora'
 'picadora de zacate' 'desgranadora' 'carretas' 'otro equipo'
 'cobertizo/galera' 'molinos' 'tanques' 'pozos' 'baÑaderos' 'silos'
 'secaderos' 'abrevaderos' 'otra instalacion']
['no' 'si']
[ nan   1.  24.   5.   2.   6.   4.   3.  10.   8.  11.   7.  12.   9.
  17.  14.  15.  30.  18.  20.  33.  13.  22.  16.   0. 110. 100.  80.
  19.  25.  21.  44.  60. 200.  63. 101.  32.  23. 126.  40.  28.  70.
  65.  29.  59.  48.  54.  58.  51. 132. 145. 112.]


In [76]:
# Filter out the records where p16g01 is "no"
p16g01 = ecv28a16[ecv28a16["p16g01"].str.lower() == "no"]
# Check whether the p16g02 of all these records is 0
all_na = p16g01["p16g02"].isna().all()
print(all_na)

True


In [77]:
import pandas as pd
from collections import OrderedDict

ecv28a16 = pd.read_csv('../intermediate_data/ECV28A16.csv', encoding='utf-8-sig')
ecv28a16.columns = ecv28a16.columns.str.lower()

ecv28a16['hhid'] = ecv28a16['hhid'].astype('Int64').astype(str)

# When cleaning p16g01 = 'no', p16g02 should be set to 0
ecv28a16['p16g01'] = ecv28a16['p16g01'].str.lower().str.strip()
ecv28a16['p16g02'] = ecv28a16.apply(
    lambda row: 0 if row['p16g01'] == 'no' else row['p16g02'],
    axis=1
)
ecv28a16['p16g02'] = ecv28a16['p16g02'].fillna(0).astype(int)

# Use OrderedDict to maintain the translation order of the facility
item_translation = OrderedDict([
    ('implementos de tiro animal', 'animal_drawn_equipment'),
    ('tractor', 'tractor'),
    ('implementos para tractor', 'tractor_equipment'),
    ('cosechadora', 'harvester'),
    ('sembradora o cultivadora', 'seeder_cultivator'),
    ('bomba de agua', 'water_pump'),
    ('camion, camioneta, jeep', 'vehicle'),
    ('animales de trabajo', 'work_animals'),
    ('bomba fumigadora', 'fumigation_pump'),
    ('planta electrica', 'generator'),
    ('equipo de riego', 'irrigation_equipment'),
    ('pequeÑas herramientas', 'small_tools'),
    ('ordeÑadora', 'milking_machine'),
    ('picadora de zacate', 'forage_chopper'),
    ('desgranadora', 'sheller'),
    ('carretas', 'carts'),
    ('otro equipo', 'other_equipment'),
    ('cobertizo/galera', 'shed'),
    ('molinos', 'mills'),
    ('tanques', 'tanks'),
    ('pozos', 'wells'),
    ('baÑaderos', 'baths'),
    ('silos', 'silos'),
    ('secaderos', 'dryers'),
    ('abrevaderos', 'watering_troughs'),
    ('otra instalacion', 'other_facility')
])

# Add English variable names
ecv28a16['item_en'] = ecv28a16['item'].map(item_translation)
ecv28a16['var_name'] = 'num_' + ecv28a16['item_en']

# pivot and aggregate into sum (for handling duplicate rows)
wide_df = ecv28a16.pivot_table(
    index='hhid',
    columns='var_name',
    values='p16g02',
    aggfunc='sum'
).reset_index()

# Construct the complete hhid (1-7276), and the completion is 0
full_hhids = pd.DataFrame({'hhid': [str(i) for i in range(1, 7277)]})
wide_df = full_hhids.merge(wide_df, on='hhid', how='left')
wide_df = wide_df.fillna(0).copy()

# set original orger
ordered_cols = ['hhid'] + ['num_' + item_translation[item] for item in item_translation]
wide_df = wide_df[ordered_cols]

wide_df['hhid_sort'] = wide_df['hhid'].astype(int)
wide_df = wide_df.sort_values('hhid_sort').drop(columns='hhid_sort')
wide_df['hhid'] = wide_df['hhid'].astype(str)

wide_df.to_csv('../cleaned_data/ECV28A16_cleaned.csv', index=False, encoding='utf-8-sig')


In [73]:
wide_df['num_harvester'].value_counts(dropna = False)

num_harvester
0.0    7276
Name: count, dtype: int64

## 13. Clean ```ECV31A16``` - Livestock Activity

In [41]:
ecv31a16 = pd.read_csv('../intermediate_data/ECV31A16.csv', encoding='utf-8-sig')
missing_report = ecv31a16.isnull().sum()
print(missing_report[missing_report>0])
print(ecv31a16.head())

Series([], dtype: int64)
   hhid      tipo                    item p16j02  p16j03
0  38.0  pecuaria  vacas, toros, terneros     no     0.0
1  38.0  pecuaria                  cabras     no     0.0
2  38.0  pecuaria                  ovejas     no     0.0
3  38.0  pecuaria                  cerdos     no     0.0
4  38.0  pecuaria                 conejos     no     0.0


In [42]:
print(ecv31a16['item'].nunique(), ecv31a16['item'].unique())
print(ecv31a16['p16j02'].unique())
print(ecv31a16['p16j03'].unique())

11 ['vacas, toros, terneros' 'cabras' 'ovejas' 'cerdos' 'conejos'
 'gallinas y pollos' 'pavos o chompipes' 'patos' 'caballos, burros, mulas'
 'colmenas' 'otros animales']
['no' 'si']
[0.000e+00 1.100e+01 1.000e+00 2.000e+00 7.000e+00 1.000e+01 4.000e+00
 8.000e+00 1.400e+01 1.600e+01 3.000e+00 1.200e+01 6.000e+00 4.500e+01
 9.250e+02 5.000e+00 2.000e+01 1.300e+01 1.500e+01 2.500e+01 3.600e+01
 3.500e+01 2.300e+01 1.700e+01 9.000e+00 5.000e+02 3.700e+01 7.500e+01
 1.800e+01 2.600e+01 4.000e+01 3.000e+01 6.500e+01 2.400e+01 6.000e+01
 2.200e+01 2.800e+01 2.700e+01 8.000e+01 4.800e+01 1.000e+03 1.800e+02
 5.000e+01 1.000e+02 3.000e+03 2.100e+01 3.500e+02 1.900e+01 3.400e+01
 3.800e+01 3.100e+01 2.900e+01 3.300e+01 7.000e+01 1.750e+02 6.700e+01
 5.500e+01 3.200e+01 2.050e+02 1.600e+02 1.300e+02 1.200e+02 2.300e+04
 2.000e+02 6.100e+01 2.500e+02 4.000e+02 4.400e+01 4.300e+01 1.200e+03
 5.200e+01 1.500e+02 6.000e+02 4.000e+03 1.500e+03 8.000e+02 4.600e+01
 4.100e+01 1.224e+04 9.000e+01 5.900

In [43]:
from collections import OrderedDict

ecv31a16.columns = ecv31a16.columns.str.lower()

ecv31a16['hhid'] = ecv31a16['hhid'].astype(pd.Int64Dtype()).astype(str)

# Use OrderedDict to maintain the translation order of animals
animal_translation = OrderedDict([
    ('vacas, toros, terneros', 'cattle'),
    ('cabras', 'goats'),
    ('ovejas', 'sheep'),
    ('cerdos', 'pigs'),
    ('conejos', 'rabbits'),
    ('gallinas y pollos', 'chickens'),
    ('pavos o chompipes', 'turkeys'),
    ('patos', 'ducks'),
    ('caballos, burros, mulas', 'horses_donkeys_mules'),
    ('colmenas', 'beehives'),
    ('otros animales', 'other_animals')
])

# 名Add English variable names
ecv31a16['item_en'] = ecv31a16['item'].map(animal_translation)
ecv31a16['var_name'] = 'num_' + ecv31a16['item_en']

# pivot to wide format
wide_df = ecv31a16.pivot_table(
    index='hhid',
    columns='var_name',
    values='p16j03',
    aggfunc='sum'
).reset_index()

# The complete hhid (1-7276) was constructed, and the completion was 0
full_hhids = pd.DataFrame({'hhid': [str(i) for i in range(1, 7277)]})
wide_df = full_hhids.merge(wide_df, on='hhid', how='left')
wide_df = wide_df.fillna(0).copy()

# clarify the column sequence (hhid 11 columns of animals)
ordered_cols = ['hhid'] + ['num_' + animal_translation[item] for item in animal_translation]
wide_df = wide_df[ordered_cols]

wide_df['hhid_sort'] = wide_df['hhid'].astype(int)
wide_df = wide_df.sort_values('hhid_sort').drop(columns='hhid_sort')
wide_df['hhid'] = wide_df['hhid'].astype(str)

wide_df.to_csv('../cleaned_data/ECV31A16_cleaned.csv', index=False, encoding='utf-8-sig')


## 14. Clean ```ECOM02``` - road availability

In [44]:
ecom02 = pd.read_csv('../intermediate_data/ECOM02.csv', encoding='utf-8-sig')
missing_report = ecom02.isnull().sum()
print(missing_report[missing_report>0])
print(ecom02.head())

Series([], dtype: int64)
   depto  mupio  sector  c02act  c02a01
0      1      5      50       1       1
1      1      5      50       2       2
2      1      5      50       3       2
3      1      5      50       4       2
4      1      5      50       5       2


In [45]:
print(consumo5['depto'].nunique(), consumo5['depto'].unique())
print(ecom02['c02act'].nunique(), ecom02['c02act'].unique())

22 ['guatemala', 'el progreso', 'sacatepequez', 'chimaltenango', 'escuintla', ..., 'izabal', 'zacapa', 'chiquimula', 'jalapa', 'jutiapa']
Length: 22
Categories (22, object): ['alta verapaz', 'baja verapaz', 'chimaltenango', 'chiquimula', ..., 'solola', 'suchitepequez', 'totonicapan', 'zacapa']
8 [1 2 3 4 5 6 7 8]


In [46]:
ecom02.dtypes

depto     int64
mupio     int64
sector    int64
c02act    int64
c02a01    int64
dtype: object

In [47]:
# Create provincial number - name mapping
province_map = {
    1: 'guatemala', 2: 'el progreso', 3: 'sacatepequez', 4: 'chimaltenango',
    5: 'escuintla', 6: 'santa rosa', 7: 'solola', 8: 'totonicapan',
    9: 'quetzaltenango', 10: 'suchitepequez', 11: 'retalhuleu', 12: 'san marcos',
    13: 'huehuetenango', 14: 'quiche', 15: 'baja verapaz', 16: 'alta verapaz',
    17: 'peten', 18: 'izabal', 19: 'zacapa', 20: 'chiquimula',
    21: 'jalapa', 22: 'jutiapa'
}
ecom02['depto_name'] = ecom02['depto'].map(province_map)

# OrderedDict ensures order
transport_translation = OrderedDict([
    (0, 'paved_or_gravel_road'),
    (1, 'dirt_road'),
    (2, 'footpath_no_gravel'),
    (3, 'footpath'),
    (4, 'sea_lake_river'),
    (5, 'train'),
    (6, 'airplane'),
    (7, 'other')
])
ecom02['transport_index'] = ecom02.groupby(['depto_name', 'mupio', 'sector']).cumcount()
ecom02['transport_en'] = ecom02['transport_index'].map(transport_translation)

# ）The presence or absence of standardized transportation facilities (si/no)
ecom02['has_transport'] = ecom02['c02a01'].astype(str).str.lower().str.strip()

# pivot to wide format
ecom02_wide = ecom02.pivot_table(
    index=['depto_name', 'mupio', 'sector'],
    columns='transport_en',
    values='has_transport',
    aggfunc='first'
).reset_index()

# rename to has_<type>
ecom02_wide.columns = ['depto', 'mupio', 'sector'] + [f'has_{col}' for col in ecom02_wide.columns[3:]]

# prepare consumo5
consumo5['hhid'] = consumo5['hhid'].astype(pd.Int64Dtype()).astype(str)
consumo5['depto'] = consumo5['depto'].str.strip().str.lower()
consumo5['mupio'] = consumo5['mupio'].astype(int)
consumo5['sector'] = consumo5['sector'].astype(int)

# merge with transport data
merged = consumo5.merge(
    ecom02_wide,
    on=['depto', 'mupio', 'sector'],
    how='left'
)

# missing
transport_cols = [col for col in merged.columns if col.startswith('has_')]
merged[transport_cols] = merged[transport_cols].fillna('missing')

# ensure order
transport_cols_ordered = [f'has_{v}' for v in transport_translation.values()]
final_cols = ['hhid'] + transport_cols_ordered
merged = merged[final_cols]

merged.to_csv('../cleaned_data/ECOM02_cleaned.csv', index=False, encoding='utf-8-sig')


In [48]:
# province_map（name → code）
province_code_map = {v: k for k, v in province_map.items()}

# SExtract community identification information
community_info = consumo5[['hhid', 'depto', 'mupio', 'sector']].copy()
community_info['depto_code'] = community_info['depto'].map(province_code_map)
community_info['depto_name'] = community_info['depto']

# reorder
community_info = community_info[['hhid', 'depto_code', 'depto_name', 'mupio', 'sector']]

community_info.to_csv('../cleaned_data/geo_info_per_household.csv', index=False, encoding='utf-8-sig')


## 15. Clean ```ECOM03``` - infrastructure availablity

In [49]:
ecom03 = pd.read_csv('../intermediate_data/ECOM03.csv', encoding='utf-8-sig')

missing_report = ecom03.isnull().sum()
print("Missing report:")
print(missing_report[missing_report>0])

ecom03

Missing report:
c02b01      351
c02b02a    1597
c02b02b    1535
dtype: int64


,depto,mupio,sector,c02b01,c02b02a,c02b02b
0,1,5,50,1.0,1.0,2.0
1,1,5,50,1.0,1.0,2.0
2,1,5,50,1.0,1.0,2.0
3,1,5,50,1.0,1.0,2.0
4,1,5,50,1.0,1.0,2.0
...,...,...,...,...,...,...
11949,22,13,4,2.0,4.0,2.0
11950,22,13,4,2.0,70.0,2.0
11951,22,13,4,2.0,2.0,2.0
11952,22,13,4,1.0,NaN,NaN


In [50]:
ecom03['c02b02b'].value_counts()

c02b02b
2.0    7225
9.0    1162
1.0    1000
3.0     967
4.0      65
Name: count, dtype: int64

In [51]:
from collections import OrderedDict

facility_dict = OrderedDict({
    1: "preschool",
    2: "primary_school",
    3: "secondary_school",
    4: "health_center",
    5: "public_hospital",
    6: "private_hospital",
    7: "private_clinic",
    8: "natural_healer",
    9: "traditional_midwife",
    10: "pharmacy",
    11: "public_phone",
    12: "mail_service",
    13: "bus_stop",
    14: "bank",
    15: "cooperative",
    16: "police_post",
    17: "civil_registry",
    18: "market",
    19: "church",
    20: "community_room",
    21: "recreation_parks",
    22: "firehouse",
    23: "firewood_collection",
    24: "water_collection",
    25: "work_site"
})


In [52]:
unit_map = {
    1: 1,
    2: 1000,
    3: 100,
    4: 5572,
    9: None,
    None: None
}

In [53]:
geo = pd.read_csv('../cleaned_data/geo_info_per_household.csv', encoding='utf-8-sig')

geo

,hhid,depto_code,depto_name,mupio,sector
0,1,1,guatemala,3,3
1,2,1,guatemala,3,3
2,3,1,guatemala,3,3
3,4,1,guatemala,3,3
4,5,1,guatemala,3,3
...,...,...,...,...,...
7271,7272,22,jutiapa,13,7
7272,7273,22,jutiapa,13,7
7273,7274,22,jutiapa,13,7
7274,7275,22,jutiapa,13,7


In [54]:
ecom03['c02b02b'] = pd.to_numeric(ecom03['c02b02b'], errors='coerce')  # ensures float dtype

# Add the facility number (exactly 25 lines for each community)
ecom03['infra_index'] = ecom03.groupby(['depto', 'mupio', 'sector']).cumcount() + 1


In [55]:
ecom03['c02b02b'] 

0        2.0
1        2.0
2        2.0
3        2.0
4        2.0
        ... 
11949    2.0
11950    2.0
11951    2.0
11952    NaN
11953    2.0
Name: c02b02b, Length: 11954, dtype: float64

In [56]:
ecom03['unit_multiplier'] = ecom03['c02b02b'].map(unit_map)

ecom03['distance_missingness'] = ecom03['c02b02a'].isna() | ecom03['unit_multiplier'].isna()

# Compute distance
ecom03['distance_m'] = ecom03['c02b02a'] * ecom03['unit_multiplier']
impute_mean_dist = ecom03['distance_m'].mean()
ecom03['distance_m'] = ecom03['distance_m'].fillna(impute_mean_dist)  

ecom03


,depto,mupio,sector,c02b01,c02b02a,c02b02b,infra_index,unit_multiplier,distance_missingness,distance_m
0,1,5,50,1.0,1.0,2.0,1,1000.0,False,1000.000000
1,1,5,50,1.0,1.0,2.0,2,1000.0,False,1000.000000
2,1,5,50,1.0,1.0,2.0,3,1000.0,False,1000.000000
3,1,5,50,1.0,1.0,2.0,4,1000.0,False,1000.000000
4,1,5,50,1.0,1.0,2.0,5,1000.0,False,1000.000000
...,...,...,...,...,...,...,...,...,...,...
11949,22,13,4,2.0,4.0,2.0,21,1000.0,False,4000.000000
11950,22,13,4,2.0,70.0,2.0,22,1000.0,False,70000.000000
11951,22,13,4,2.0,2.0,2.0,23,1000.0,False,2000.000000
11952,22,13,4,1.0,NaN,NaN,24,NaN,True,27994.267603


In [57]:
# Retain the designated facilities
subset_keys = list(facility_dict.keys())
keep_keys = subset_keys[:15] + [18]
ecom03 = ecom03[ecom03['infra_index'].isin(keep_keys)]

In [58]:
# add cols
ecom03['facility'] = ecom03['infra_index'].map(facility_dict)
ecom03['distance_col'] = 'distance_' + ecom03['facility']

# Pivot to wide format
distance_wide = ecom03.pivot_table(
    index=['depto', 'mupio', 'sector'],
    columns='distance_col',
    values='distance_m',
    aggfunc='first'
).reset_index()


C:\Users\PB\AppData\Local\Temp\ipykernel_7772\360888750.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ecom03['facility'] = ecom03['infra_index'].map(facility_dict)
C:\Users\PB\AppData\Local\Temp\ipykernel_7772\360888750.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ecom03['distance_col'] = 'distance_' + ecom03['facility']


In [59]:
# standardize geo_info
geo['depto'] = geo['depto_code'].astype(int)
geo['mupio'] = geo['mupio'].astype(int)
geo['sector'] = geo['sector'].astype(int)

# Merge the distance of community infrastructure
merged = geo.merge(distance_wide, on=['depto', 'mupio', 'sector'], how='left')

# change missing to 99999999
distance_cols = [f'distance_{facility_dict[k]}' for k in keep_keys]
merged[distance_cols] = merged[distance_cols].fillna(99999999)

ordered_cols = (
    ['hhid'] +
    distance_cols
)
merged = merged[ordered_cols]


In [60]:
merged.to_csv('../cleaned_data/ECOM03_cleaned.csv', index=False, encoding='utf-8-sig')


## 16. Merge all data based on ```hhid```

In [61]:
import os
import pandas as pd

# path storing cleaned data
cleaned_path = '../cleaned_data'
files = [f for f in os.listdir(cleaned_path) if f.endswith('.csv')]

hhid_sets = {}
row_counts = {}

for file in files:
    path = os.path.join(cleaned_path, file)
    try:
        df = pd.read_csv(path, encoding='utf-8-sig')
    except Exception as e:
        print(f"[ERROR] Failed to read {file}: {e}")
        continue

    if 'hhid' not in df.columns:
        print(f"[MISSING] 'hhid' column not found in {file}")
        continue

    hhid_list = df['hhid'].astype(str).tolist()
    hhid_sets[file] = set(hhid_list)
    row_counts[file] = len(hhid_list)

# 1. check rows num
print("\nChecking row counts...")
base_file, base_count = next(iter(row_counts.items()))
for file, count in row_counts.items():
    if count != base_count:
        print(f"[ROW COUNT MISMATCH] {file}: {count} rows (expected {base_count})")

# 2. check hhid
print("\nChecking hhid consistency...")
base_hhids = hhid_sets[base_file]
for file, hhids in hhid_sets.items():
    if hhids != base_hhids:
        missing = base_hhids - hhids
        extra = hhids - base_hhids
        print(f"[HHID MISMATCH] {file}:")
        if missing:
            print(f"  ↳ Missing hhids (vs base): {sorted(list(missing))[:5]}...")
        if extra:
            print(f"  ↳ Extra hhids (not in base): {sorted(list(extra))[:5]}...")

print("\nConsistency check complete.")



Checking row counts...

Checking hhid consistency...

Consistency check complete.


In [62]:

cleaned_path = '../cleaned_data'
merged_path = '../merged_data'
all_files = [f for f in os.listdir(cleaned_path) if f.endswith('.csv')]

merge_files = [f for f in all_files if f != 'geo_info_per_household.csv']

# main table: HOGARES_cleaned.csv
merge_files_sorted = ['HOGARES_cleaned.csv'] + [f for f in merge_files if f != 'HOGARES_cleaned.csv']
main_df = pd.read_csv(os.path.join(cleaned_path, merge_files_sorted[0]), encoding='utf-8-sig')

# all left joint
for file in merge_files_sorted[1:]:
    path = os.path.join(cleaned_path, file)
    df = pd.read_csv(path, encoding='utf-8-sig')
    print(f"Merging: {file} ({df.shape[1]} columns)")
    main_df = main_df.merge(df, on='hhid', how='left')

output_path = os.path.join(merged_path, 'guatemala_household_data.csv')
main_df.to_csv(output_path, index=False, encoding='utf-8-sig')

print(f"\nFinal merged dataset saved to: {output_path}")
print(f"Final shape: {main_df.shape[0]} rows, {main_df.shape[1]} columns")


Merging: AUTOIDH_cleaned.csv (3 columns)
Merging: CONSUMO5_cleaned.csv (6 columns)
Merging: ECOM02_cleaned.csv (9 columns)
Merging: ECOM03_cleaned.csv (17 columns)


Merging: ECV01H01_cleaned.csv (29 columns)
Merging: ECV02H01_cleaned.csv (22 columns)
Merging: ECV09P05_cleaned.csv (15 columns)
Merging: ECV11P10_cleaned.csv (6 columns)
Merging: ECV17E14_cleaned.csv (39 columns)
Merging: ECV18N15_cleaned.csv (3 columns)
Merging: ECV21A16_cleaned.csv (2 columns)
Merging: ECV28A16_cleaned.csv (27 columns)
Merging: ECV31A16_cleaned.csv (12 columns)

Final merged dataset saved to: ../merged_data\guatemala_household_data.csv
Final shape: 7276 rows, 182 columns


In [63]:
# check missing
merged_path = '../merged_data/guatemala_household_data.csv'
merged_df = pd.read_csv(merged_path, encoding='utf-8-sig')

total_missing = merged_df.isnull().sum().sum()
if total_missing == 0:
    print("No missing values in the merged dataset.")
else:
    print(f"Total missing values in dataset: {total_missing}")
    print("\n Missing value count per column (non-zero only):")
    missing_per_column = merged_df.isnull().sum()
    print(missing_per_column[missing_per_column > 0].sort_values(ascending=False))

    print("\nTop 10 columns with most missing values:")
    print(missing_per_column.sort_values(ascending=False).head(10))


No missing values in the merged dataset.


## 17. Add ```headcount_adjusted_hh_wgt```

In [64]:
# create headcount-adjusted weight col
merged_df['headcount_adjusted_hh_wgt'] = merged_df['hh_wgt'] * merged_df['hh_size']
merged_df.to_csv(merged_path, index=False, encoding='utf-8-sig')

## 18. Generate ```country_name_metadata.csv```

In [65]:
def infer_data_type(series):
    if pd.api.types.is_numeric_dtype(series):
        return 'numeric'
    else:
        return 'categorical'

variable_info = pd.DataFrame({
    'variable_name': df.columns,
    'data_type': [infer_data_type(df[col]) for col in df.columns]
})

variable_info.to_csv('../metadata/final_variable_types.csv', index=False, encoding='utf-8-sig')

In [66]:
var_types_df = pd.read_csv('../metadata/final_variable_types.csv', encoding='utf-8-sig')
metadata_df = pd.read_csv('../metadata/guatemala_metadata.csv', encoding='utf-8-sig').copy()

var_types_df['variable_name'] = var_types_df['variable_name'].str.strip().str.lower()
metadata_df['variable_name'] = metadata_df['variable_name'].str.strip().str.lower()

merged_df = var_types_df.merge(metadata_df, on='variable_name', how='left')

merged_df.to_csv('../metadata/final_variable_types.csv', index=False, encoding='utf-8-sig')

In [67]:
# Amend geeographic indicator
metadata_df['geographic_indicator'] = metadata_df['geographic_indicator'].map({'no' : False, 'yes' : True})

# Formatting
metadata_df.drop(metadata_df[metadata_df['variable_name'] == 'agreg3'].index, inplace=True)
metadata_df['variable_name'] = metadata_df['variable_name'].replace('thogar', 'hh_size')

In [68]:
metadata_df.to_csv('../metadata/guatemala_metadata_final.csv', index=False, encoding='utf-8-sig')